# Building and Training a Neural Network with FFBP
This notebook provides instructions on how to build and train your own neural network using pdpyflow's FFBP package and Tensorflow. The FFBP package itself runs tensorflow underneath and is intended to simplify the process of [Tensorflow Graph](https://www.tensorflow.org/programmers_guide/graphs) construction. Tensorflow graph is a computational structure that *describes* the flow of data (tensors) through various computational operations. Thus, constructing and running a graph are separated conceptually, as well as in the code. 

Training a neural network is accomplished in three steps:
1. Defining input data [&#x21F1;](#step1)
2. Constructing neural network [&#x21F1;](#step2)
3. Running neural network [&#x21F1;](#step3)

We begin by importing the required packages. Run the cell below to make the imports (<kbd>Ctrl</kbd>+<kbd>Enter</kbd>).

In [ ]:
import tensorflow as tf
import FFBP
tf.logging.set_verbosity(tf.logging.ERROR) # Prevent unwanted logging messages by tensorflow

<a id='step2'> </a>
## 1. Defining Input Data
Next, we need to create `FFBP.InputData` for training and testing. Refer to the [InputData tutorial](https://github.com/alex-ten/pdpyflow/blob/master/tutorials/FFBP_network/InputData_tutorial.ipynb) to see how to create an `InputData` object. The cell below creates two `InputData` objects, `TRAIN_DATA` and `TEST_DATA` that will be used for training and testing, respectively. Note, by convention we capitalize the names of variables that are referenced globally accross cells (e.g. `FFBP_GRAPH`, `NUM_EPOCHS`).

In [ ]:
NUM_EPOCHS = 2

FFBP_GRAPH = tf.Graph()

# Create FFBP.InputData for training and testing (within the context of created graph)
with FFBP_GRAPH.as_default():
    
    # Create data for training
    TRAIN_DATA = FFBP.InputData(
        path_to_data_file = 'auto_data_train.txt',
        num_epochs = NUM_EPOCHS,
        batch_size = 4,
        data_len = 8,
        inp_size = 8, 
        targ_size = 8,
        shuffle_seed = SHUFFLE
    )
    # Create data for testing
    TEST_DATA = FFBP.InputData(
        path_to_data_file = 'auto_data_test.txt',
        num_epochs = NUM_EPOCHS,
        batch_size = 15,
        inp_size = 8, 
        targ_size = 8,
        data_len = 15
    )

<a id='step1'> </a>
## 2. Constructing Neural Network
Before we create a network (`FFBP.Model` object) we need to construct its components:
- Input placeholder
- Network layers
- Target placeholder


In [ ]:
# Add network components to the graph
with FFBP_GRAPH.as_default():
    model_name = 'xor_model'
    with tf.name_scope(model_name):
        
        # Create an input placeholder
        input_  = tf.placeholder(dtype = tf.float32, shape=[None, INP_SIZE], name='model_inp')
        
        # Create first hidden layer
        hidden_layer = FFBP.BasicLayer(
            layer_name = 'hidden_layer', 
            layer_input = input_, 
            size = 2, 
            wrange = [-WR, WR], 
            nonlin = tf.nn.sigmoid, 
            bias = True, 
            seed = None
        )
        
        # Create another first-level hidden layer
        output_layer = FFBP.BasicLayer(
            layer_name = 'output_layer', 
            layer_input = hidden_layer.output, 
            size = 1, 
            wrange = [-WR, WR], 
            nonlin = tf.nn.sigmoid, 
            bias = True, 
            seed = None
        )

        target = tf.placeholder(dtype = tf.float32, shape=[None, TARG_SIZE], name='targets')

        MODEL = FFBP.Model(
            name = model_name,
            layers = [hidden_layer, output_layer],
            train_data = train_data, 
            inp        = input_,
            targ       = target,
            loss       = tf.reduce_sum(tf.squared_difference(target, output_layer.output), name='loss_function'),
            optimizer  = tf.train.MomentumOptimizer(lr, m),
            test_data  = test_data
        )